# Assignment-1: Transformations and representations

Team Name: Bhagwaan Bharose

Roll number: 2019111019, 2019111026

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `Set Up` for detailed step-by-step instructions about the installation setup.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. 
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 11/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.


# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

# Set Up

We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. All assignments will be python based, hence familiarising yourself with Python is essential.


## Setting up Anaconda environment (Recommended)

1. Install Anaconda or Miniconda from [here](https://docs.conda.io/projects/conda/en/latest/user-guide/install/linux.html) depending on your requirements.
2. Now simply run `conda env create -f environment.yml` in the current folder to create an environment `mr_assignment1` (`environment.yml` can be found in `misc/`).
3. Activate it using `conda activate mr_assignment1`.

## Setting up Virtual environment using venv

You can also set up a virtual environment using venv

1. Run `sudo apt-get install python3-venv` from command line.
2. `python3 -m venv ~/virtual_env/mr_assignment1`. (you can set the environment path to anything)
3. `source ~/virtual_env/mr_assignment1/bin/activate`
4. `pip3 install -r requirements.txt` from the current folder (`requirements.txt` can be found in `misc/`).

In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1. Getting started with Open3D

Open3D is an open-source library that deals with 3D data, such as point clouds, mesh. We'll be using Open3D frequently as we work with point clouds. Let's start with something simple:

<img src="misc/bunny.jpg" alt="drawing" width="200"/>

1. Read the Stanford Bunny file (in `data/`) given to you and visualise it using Open3D.
2. Convert the mesh to a point cloud and change the colour of points.
3. Set a predefined viewing angle (using Open3D) for visualization and display the axes while plotting.
4. Scale, Transform, and Rotate the rabbit (visualise after each step).
5. Save the point cloud as bunny.pcd.

In [3]:
# Part 1 - Read the Stanford Bunny file (in data/) given to you and visualise it using Open3D.
mesh = o3d.io.read_triangle_mesh("./data/bunny.ply")
o3d.visualization.draw_geometries([mesh])
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

# Part 2 - Convert the mesh to a point cloud and change the colour of points.
pcd = o3d.geometry.PointCloud()
pcd.points = mesh.vertices
pcd.colors = mesh.vertex_colors
pcd.normals = mesh.vertex_normals
o3d.visualization.draw_geometries([pcd])
pcd.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcd])

# Part 3 - Set a predefined viewing angle (using Open3D) for visualization and display the axes while plotting.
FOR = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.025, origin=[0,0,0])
o3d.visualization.draw_geometries([pcd, FOR], 
                                  zoom=1, 
                                  front=[-0.01, 0.01, 0], 
                                  lookat=[-0.01, 0.01, 0],
                                  up=[0,1,0]
                                 )

# Part 4 - Scale, Transform, and Rotate the rabbit (visualise after each step).

#Scale
pcd.scale(1.5, center=pcd.get_center())
o3d.visualization.draw_geometries([pcd, FOR], 
                                  zoom=1, 
                                  front=[-0.01, 0.01, 0], 
                                  lookat=[-0.01, 0.01, 0],
                                  up=[0,1,0]
                                 )
# Transform
T = np.eye(4)
T[:3, :3] = pcd.get_rotation_matrix_from_xyz((0, np.pi / 3, np.pi / 2))
T[0, 3] = 0.1
T[1, 3] = 0.1
pcd = pcd.transform(T)
o3d.visualization.draw_geometries([pcd, FOR], 
                                  zoom=1, 
                                  front=[-0.01, 0.01, 0], 
                                  lookat=[-0.01, 0.01, 0],
                                  up=[0,1,0]
                                 )
# Rotate
R = pcd.get_rotation_matrix_from_xyz((np.pi/2, 0, 0))
pcd.rotate(R, center=(0, 0, 0))
o3d.visualization.draw_geometries([pcd, FOR])

# Part 5 - Save the point cloud as bunny.pcd.
o3d.io.write_point_cloud("./results/1/5/bunny.pcd", pcd) # result is being "./results/1/5/bunny.pcd"

True

# 2. Transformations and representations

## a) Euler angles
1. Write a function that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians (X-Y-Z)

2. Solve for angles using ```fsolve from scipy``` for three initializations of your choice and compare.
$$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] 
$$

$$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] 
$$

3. What is a Gimbal lock? 

4. Show an example where a Gimbal lock occurs and visualize the Gimbal lock on the given bunny point cloud. You have to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*


In [85]:
import numpy as np
import math

def rotation_matrix(theta):
    R_x = np.array([[1, 0, 0],
                    [0, math.cos(theta[0]), -math.sin(theta[0])],
                    [0, math.sin(theta[0]), math.cos(theta[0])]
                   ])
    R_y = np.array([[math.cos(theta[1]), 0, math.sin(theta[1])],
                    [0, 1, 0],
                    [-math.sin(theta[1]), 0, math.cos(theta[1])]
                    ])
    R_z = np.array([[math.cos(theta[2]), -math.sin(theta[2]), 0],
                    [math.sin(theta[2]), math.cos(theta[2]), 0],
                    [0, 0, 1]
                    ])
    gg = np.dot(np.dot(R_x,R_y), R_z)
    return gg.round(15) # Remove `.round(15)` to get exact calculation done by numpy 

#Set the angles here
alpha=0
beta=np.pi/2
gamma=0

R=rotation_matrix([alpha, beta, gamma])
print("R = ", R)

R =  [[ 0.  0.  1.]
 [ 0.  1.  0.]
 [-1.  0.  0.]]


Here, if A represents the fixed frame and B represents the rotating frame then the first rotation happens about X<sub>B</sub> by α, then about Y<sub>B</sub> by β, and finally about Z<sub>B</sub> by γ. In the above output, the rotation matrix is parameterized by X-Y-Z Euler angles will be indicated by the notation <sup>A</sup>R<sub>B</sub> <sub>XYZ</sub> (α, β, γ). 
Thinking of the rotations as descriptions of these frames, we can immediately write <sup>A</sup>R<sub>B</sub> = <sup>A</sup>R<sub>B1</sub> * <sup>B1</sup>R<sub>B2</sub> * <sup>B2</sup>R<sub>B3</sub>.

In [98]:
from scipy.optimize import fsolve

def rotation_equations(thetas):
    ca=np.cos(thetas[0])
    cb=np.cos(thetas[1])
    cc=np.cos(thetas[2])
    sa=np.sin(thetas[0])
    sb=np.sin(thetas[1])
    sc=np.sin(thetas[2])
    return [
        -cb*sc - Mat[0][1],
        -sa*sb*sc+ca*cc - Mat[1][1],
        ca*sb*sc+sa*cc - Mat[2][1]
    ]

M=[[0.26200263,-0.19674724,0.944799],
[0.21984631,0.96542533,0.14007684],
[-0.93969262,0.17101007,0.29619813]]

N=[[0,-0.173648178,0.984807753],
[0,0.984807753,0.1736448178],
[-1,0,0]]

# Solving angles for matrix M using different initialisations
print("\nAngles for matrix M (in radians):")
Mat = np.array(M)
x = fsolve(rotation_equations, [-0.45,1.25,0.65])
print((x%np.pi))
x = fsolve(rotation_equations, [5.88,7.52,6.95])
print((x%np.pi))
x = fsolve(rotation_equations, [5.85, 1.24, 0.64])
print((x%np.pi))

# Solving angles for matrix M using different initialisations
print("\nAngles for matrix N (in radians):")
Mat = np.array(N)
x = fsolve(rotation_equations, [-1.6,1.4,1.6])
print((x%np.pi))
x = fsolve(rotation_equations, [-1.57,1.4,1.57]) # can guess 1.57 due to structure of the matrix
print((x%np.pi))
x = fsolve(rotation_equations, [-1.57, 1.3, 1.57])
print((x%np.pi))


Angles for matrix M (in radians):
[2.69092136 1.24081926 0.6525376 ]
[2.69482461 1.23915393 0.64884598]
[2.71033706 1.23232451 0.63419604]

Angles for matrix N (in radians):
[1.54064068 1.39618562 1.6004937 ]
[1.57227972 1.39626321 1.56933547]
[1.22823297 1.38573419 1.90794697]


After choosing 3 initializations to get almost equal values, we can now take average of these angles and generate rotation matrices for them to see how close we are.

In [105]:
# Checking for M
angles1 = np.array([2.69092136, 1.24081926, 0.6525376 ])
angles2 = np.array([2.69482461, 1.23915393, 0.64884598])
angles3 = np.array([2.71033706, 1.23232451, 0.63419604])
angles = (angles1+angles2+angles3)/3
print("\nMatrix M:\n",rotation_matrix(angles))

# Checking for N
angles1 = np.array([1.54064068, 1.39618562, 1.6004937 ])
angles2 = np.array([1.57227972, 1.39626321, 1.56933547])
angles3 = np.array([1.22823297, 1.38573419, 1.90794697])
angles = (angles1+angles2+angles3)/3
print("\nMatrix N:\n",rotation_matrix(angles))


Matrix M:
 [[ 0.26144617 -0.19677674  0.94494699]
 [-0.21976879 -0.96542005 -0.14023486]
 [ 0.93986573 -0.17100599 -0.29565076]]

Matrix N:
 [[-2.15202677e-02 -1.75816937e-01  9.84187626e-01]
 [ 3.85591372e-03 -9.84422879e-01 -1.75774649e-01]
 [ 9.99760976e-01  1.22250487e-05  2.18629789e-02]]


As we can see up on comparing with the original values, the matrices we have obtained are roughly equal.

**What is a Gimbal lock?** 

Gimbal lock is a state that we may reach while performing rotation using Euler angles, purely because how we mathematically paramaterised our rotations. Let the angles during X-Y-Z Euler angle rotation be θ<sub>x</sub>, θ<sub>y</sub> and θ<sub>z</sub>, then we may reach a state where there is no way to rotate around one of the 3 axis (that is, loss of one degree of freedom).

For example in X-Y-Z euler angle rotation, if θ<sub>y</sub> is set as π/2, then no matter how we adjust θ<sub>x</sub> and θ<sub>z</sub>, we are locked into a single axis of rotation. The rotation matrix after putting θ<sub>y</sub> = π/2 is shown below:

<img src="H2.png" alt="drawing" width="300"/>

In [3]:
# Visualizing gimbal lock by example on bunny point cloud

#Getting the bunny
mesh = o3d.io.read_triangle_mesh("./data/bunny.ply")
pcd = o3d.geometry.PointCloud()
pcd.points = mesh.vertices
pcd.colors = mesh.vertex_colors
pcd.normals = mesh.vertex_normals
pcd.paint_uniform_color([1, 0, 0])

# Example of gimbal lock has already been mentioned earlier, if we rotate about Y axis by pi/2
# then we will be stuck in an axis and rotation about X and Z won't matter.

# Plotting the basic point cloud once and then shiting it to origin for easier rotation visualizaion
FOR = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0,0,0])
pcd = pcd.translate((0.02675991, -0.09521606, -0.00894711))
R = pcd.get_rotation_matrix_from_xyz((0, np.pi, 0))
pcd.rotate(R, center=(0, 0, 0))
R = pcd.get_rotation_matrix_from_xyz((np.pi/2, 0, 0))
pcd.rotate(R, center=(0, 0, 0))
o3d.visualization.draw_geometries([pcd, FOR], 
                                  zoom=1, 
                                  front=[0.01, 0, 0], 
                                  lookat=[0.01, 0, 0],
                                  up=[0,0,1]
                                 )

vis = o3d.visualization.Visualizer()
vis.create_window()

def euler_rot(theta):
    
    global vis
    vis.clear_geometries()
    
    source = pcd
    target = FOR
#     source = source_raw.voxel_down_sample(voxel_size=0.001)
#     target = target_raw.voxel_down_sample(voxel_size=0.001)

    R = pcd.get_rotation_matrix_from_xyz((theta[0], theta[1], theta[2]))
    pcd.rotate(R, center=(0, 0, 0))
#     vis = o3d.visualization.Visualizer()
#     vis = o3d.visualization.draw_geometries([pcd, FOR], 
#                                       zoom=1, 
#                                       front=[0.01, 0, 0], 
#                                       lookat=[0.01, 0, 0],
#                                       up=[0,0,1]
#                                      )
#     vis.create_window()
    vis.add_geometry(source)
    vis.add_geometry(target)
#     for i in range(5):
#         vis.clear_geometries()
    vis.update_geometry(source)
    vis.update_geometry(target)
    vis.poll_events()
    vis.update_renderer()
    
########################################################################    
# Showing one iteration for Gimbal lock example
for i in range(100):
    euler_rot([np.pi/2/100, 0, 0])
# vis.destroy_window()

for i in range(100):
    euler_rot([0, 0, np.pi/2/100])
# vis.destroy_window()

for i in range(100):
    euler_rot([0, np.pi/2/100, 0])
vis.destroy_window()
o3d.visualization.draw_geometries([pcd, FOR])

########################################################################
# # Showing another iteration for Gimbal lock example
# pcd = o3d.geometry.PointCloud()
# pcd.points = mesh.vertices
# pcd.colors = mesh.vertex_colors
# pcd.normals = mesh.vertex_normals
# pcd.paint_uniform_color([1, 0, 0])
# pcd = pcd.translate((0.02675991, -0.09521606, -0.00894711))
# R = pcd.get_rotation_matrix_from_xyz((0, np.pi, 0))
# pcd.rotate(R, center=(0, 0, 0))
# R = pcd.get_rotation_matrix_from_xyz((np.pi/2, 0, 0))
# pcd.rotate(R, center=(0, 0, 0))

# for i in range(6):
#     euler_rot([np.pi/6, 0, 0])
# for i in range(6):
#     euler_rot([0, -np.pi/2/6, 0])
# for i in range(6):
#     euler_rot([np.pi/2/6, 0, 0])

########################################################################  
# # Showing another iteration for Gimbal lock example
# pcd = o3d.geometry.PointCloud()
# pcd.points = mesh.vertices
# pcd.colors = mesh.vertex_colors
# pcd.normals = mesh.vertex_normals
# pcd.paint_uniform_color([1, 0, 0])
# pcd = pcd.translate((0.02675991, -0.09521606, -0.00894711))
# R = pcd.get_rotation_matrix_from_xyz((0, np.pi, 0))
# pcd.rotate(R, center=(0, 0, 0))
# R = pcd.get_rotation_matrix_from_xyz((np.pi/2, 0, 0))
# pcd.rotate(R, center=(0, 0, 0))
   
# for i in range(6):
#     euler_rot([0, 0, 0])
# for i in range(6):
#     euler_rot([0, -np.pi/2/6, 0])
# for i in range(6):
#     euler_rot([np.pi/2/6, 0, 0])
    
########################################################################

# import time
# time.sleep(2)
# def euler_rot(theta):
#     R = pcd.get_rotation_matrix_from_xyz((theta[0], theta[1], theta[2]))
#     pcd.rotate(R, center=(0, 0, 0))
#     o3d.visualization.draw_geometries([pcd, FOR], 
#                                       zoom=1, 
#                                       front=[0.01, 0, 0], 
#                                       lookat=[0.01, 0, 0],
#                                       up=[0,0,1]
#                                      )  

**Example**  
Example of gimbal lock has already been mentioned earlier, if we rotate about Y axis by pi/2 then we will be stuck in an axis and rotation about X and Z won't matter.

**Visualizing this example in the bunny animation**  
If noticed carefully, in the final orientations of all the iterations, it is seen that the bunny always faces the +ve or -ve y-axis, so if the bunny is cut in a plane parallel to his front and back, that plane will always be parallel to the X-Z plane, which shows the loss in one degree of freedom.

## b) Quaternions

1. What makes Quaternions popular in graphics? 
2. Convert a rotation matrix to quaternion and vice versa. Do not use inbuilt libraries for this question.
3. Perform matrix multiplication of two $\mathcal{R}_{3 \times 3}$ rotation matrices and perform the same transformation in the quaternion space. Verify if the final transformation obtained in both the cases are the same.
4. Try to interpolate any 3D model (cube / bunny / not sphere obviously!!) between two rotation matrices and visualize!

The above questions require you to **code your own functions** and **only verify** using inbuilt functions.

**Answers**  
  
1. Quaternions are popular in graphics because:
* They don't have the problem of Gimbal lock which is present in the case of Euler angles because of the way we mathematically paramaterise our equations.
* They are a nice way to think about transformations, easy to code as well as debug.
* They have mild computation benefits (sometimes), so we have reduced computation and don't involve trigonometry.
* Fluency with quaternions can help lead into deeper/novel solutions to problems (for instance - surface parameterization and texture mapping).
* They help make interpolating rotations easy to compute and visualize.

In [80]:
# Converting quaternion to rotation matrix

def quaternion_to_rotm(q):
    
    m=[[0,0,0], [0,0,0], [0,0,0]]
    
    m[0][0] = q[0]**2 + q[1]**2 - q[2]**2 - q[3]**2
    m[0][1] = 2 * (q[1]*q[2] - q[0]*q[3])
    m[0][2] = 2 * (q[0]*q[2] + q[1]*q[3])
    m[1][0] = 2 * (q[0]*q[3] + q[1]*q[2])
    m[1][1] = q[0]**2 - q[1]**2 + q[2]**2 - q[3]**2
    m[1][2] = 2 * (q[2]*q[3] - q[0]*q[1])
    m[2][0] = 2 * (q[1]*q[3] - q[0]*q[2])
    m[2][1] = 2 * (q[0]*q[1] + q[2]*q[3])
    m[2][2] = q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2
    
    return m
    
# Set quaternion here
q = [0.707,0,0,0.707]
print("Rotation matrix = \n", np.array(quaternion_to_rotm(q)))   


Rotation matrix = 
 [[ 0.       -0.999698  0.      ]
 [ 0.999698  0.        0.      ]
 [ 0.        0.        0.999698]]


In [81]:
# Converting rotation matrix to quaternion

def rotm_to_quaternion(mat):
    
    q = [0,0,0,0]
    q[0] = np.sqrt((np.trace(mat)+1)/4) 
    q[1] = np.sqrt(mat[0][0]/2 + (1-np.trace(mat))/4) 
    q[2] = np.sqrt(mat[1][1]/2 + (1-np.trace(mat))/4)
    q[3] = np.sqrt(mat[2][2]/2 + (1-np.trace(mat))/4)

    return q
    
# Set rotation matrix here
mat = [[0,-1,0],[1,0,0],[0,0,1]]
print("Quaternions = \n", np.array(rotm_to_quaternion(np.array(mat))))   

Quaternions = 
 [0.70710678 0.         0.         0.70710678]


In [124]:
# Perform matrix multiplication of two 3×3 rotation matrices and perform the same transformation 
# in the quaternion space. Verify if the final transformation obtained in both the cases are the same.

def conjugate(q):
    return [q[0], -q[1], -q[2], -q[3]]

def multiply_q(q, p):
    res=[0,0,0,0]
    res[0] = q[0]*p[0] - q[1]*p[1] - q[2]*p[2] - q[3]*p[3]
    res[1] = q[0]*p[1] + q[1]*p[0] + q[2]*p[3] - q[3]*p[2]
    res[2] = q[0]*p[2] + q[2]*p[0] - q[1]*p[3] + q[3]*p[1]
    res[3] = q[0]*p[3] + q[3]*p[0] + q[1]*p[2] - q[2]*p[1]
    return res

# Set parameters here
q = [0.70710678, 0., 0. ,0.70710678]
mat = [[0., -0.999698, 0.], [0.999698, 0., 0.], [ 0., 0., 0.999698]]
pnt = [1,0,0]

q_con = conjugate(q)
mat=np.array(mat)
aug_pnt = [0]+pnt
pnt = np.array(pnt)

print("\nPoint after rotation using rotation matrix:\n",np.dot(mat,pnt))

res=multiply_q(multiply_q(q, aug_pnt), q_con)
res.pop(0)
print("\nPoint after rotation using quaternion:\n",res) # Performing the rotation using quaternions


Point after rotation using rotation matrix:
 [0.       0.999698 0.      ]

Point after rotation using quaternion:
 [0.0, 0.9999999966439369, 0.0]


We can see that the points above match

In [131]:
# Try to interpolate any 3D model (cube / bunny / not sphere obviously!!) between 
# two rotation matrices and visualize!

FOR = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.025, origin=[0,0,0])

q_start = [0.70710678, 0., 0. ,0.70710678]
# q_end = 

scale = 10
Ry = Rz = np.eye(3)
end=None
vec=None
T = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
T[:3, -1] = [0,0,0]
if end is not None:
    vec = np.array(end) - np.array([0,0,0])
elif vec is not None:
    vec = np.array(vec)
if end is not None or vec is not None:
    scale = vector_magnitude(vec)
    Rz, Ry = calculate_zy_rotation_for_arrow(vec)
mesh = create_arrow(scale)
# Create the arrow
mesh.rotate(Ry, center=np.array([0, 0, 0]))
mesh.rotate(Rz, center=np.array([0, 0, 0]))
mesh.translate(origin)
draw_geometries([mesh])

NameError: name 'create_arrow' is not defined

## c) Exponential maps (Bonus)

1. What is the idea behind exponential map representation of rotation matrices?
2. Perform matrix exponentiation and obtain the rotation matrix to rotate a vector $P$ around $\omega$ for $\theta$ seconds.
$$
\omega = \begin{bmatrix}2 \\ 1 \\ 15 \end{bmatrix}
$$

$$
\theta = 4.1364
$$

3. Compute the logarithmic map (SO(3) to so(3)) of the rotation matrix to obtain the rotation vector and the angle of rotation
$$
\begin{bmatrix}
0.1 &  -0.9487 & 0.3 \\
0.9487 & 0.  & -0.3162 \\
0.3   &  0.3162  & 0.9 
\end{bmatrix}
$$
You can use inbuilt libraries **only to verify** your results.

# 3. Data representations

## a) Octomaps

1. Why is an Octomap memory efficient?
2. When do we update an Octomap and why?
3. When would you likely use an octomap instead of a point cloud?
 

**Answers**  
  
1. An octomap uses an octree data structure to store the occupancy probabilities. Octree is a hierarchial data structure for spatial subdivisions in 3 dimensions. Each node in an octree represents the space contained in  a cubic volume, usually called a voxel. Each voxel can recursively be subdivided into 8 smaller voxels till the minimum size is reached. This approach is memory efficient compared to point cloud approach as instead of dividing the entire volume into individual cells, **we divide a cell further only if we have any conflict**. Hence, if we know for a huge volume that it is unoccupied, we may not further divide it. This helps us save memory, since in the case of point cloud, we would have stored the occupancy probability of each point cloud within that volume.  
<br></br>  
2. Octomap are **built incrementally**. As the robot moves around, we get occupancy probabilities from different camera locations (get new measurements) in a local map. We then first transform this data from local map to global space and then stitch the information by updating the octomap. The update is done using the recursive method (multiplying occupancy probabilities and multiplying non-occupancy probabilities obtained from different measurements and normalizing the results to get the occupancy and non-occupancy probabilities).   
Octomap is updated because:  
1) After each measurement, as we update the octomap, we progressively become more confident that a cell is occupied or unoccupied with more certainity (not talking about dynamic obstacles).  
2) A robot can collect data only for a limited range (local space). Updating the octomap helps us to helps fit the data from local space to global map so as the robot moves around, we have a bigger picture (occupancy probabilities) of the entire global map.  
<br></br>
3. If we don't want sematic understanding of the scene (sensing and identifying the objects/obstacles in the scene) and **can work with the information whether a volume is occupied or unaoccpied**, then we can use octomaps (as it doesn't replicate the same scene in its understanding but only measures the occupancy probabilities of the voxels). Also we will use an octomap instead of a point cloud when we want the system to be **memory efficient and the access time is not a problem** for us. 

## b) Signed Distance Functions

1. How do we determine object surfaces using SDF?
2. How do we aggregate views from multiple cameras? (just a general overview is fine)
3. Which preserves details better? Voxels or SDF? Why?
4. What’s an advantage of SDF over a point cloud?


**Answers**  
1. In case of SDF, instead of storing probability of occupancy of a voxel, here we try to represent the distance to the surface. For each voxel, we try estimating how far it is from the surface. After applying SDF to each voxel, if the sign is negative, the voxel lies outside the surface, if the value is zero, it lies on the surface and if it is positive then it lies inside the surface. So **if SDF for a voxel is 0, it means that it represents the object's surface**. Also, since we have the distance as well for each voxel, **after some linear calculation we can find out the surface of the object mathematically** (diagram below). 
<img src="H1.jpg" alt="drawing" width="600"/>
<br></br>  
2. Similar to octomaps, after we have one measurement, we need to fuse the values from the next measurements, This is done using a weighted average of the measurements. For each voxel, we have a distance (D) and weight (W) with is initially 0. Now each measurement for a voxel will have a distance (d) and weight (w) associated to it. Now values for a voxel are updated as:  
* $D=\frac{DW+dw}{W+w}$   
* W=W+w  
Updating stategies can differ from implementation to implementation. As we keep on aggregating views from multiple cameras, W keeps on increasing and hence we will get more confident (since this means we have more measurements). So we can use the value of W as confidence.
<br></br>  
3. SDF should preserve details better as during fusing different measurements, here we can take a custom weighted approach (that is, weighing strategy depends on implementation). Precise weighing based on confidence of measurement can help us presearve details better. So we can give higher weights to positions we are sure of getting good measurement and low weight to positions where consifdence of measurement is low. On the other hand, during fusing occupancy probabilities in voxels, all measurements get equal weightage in the recursive formula. Hence good and bad measurements influence the answer equally. Also, in SDF we are able to capture more information compared to occupancy probability of voxel.   
<br></br>  
4. Advantages of SDF over point clouds are:
* SDF gives us 2 values, weights and distance to the surface wheres point clouds only store occupancy probability. Hence in SDF we can tell the confidence of a voxel, its position (inside, on or outside the surface) and its distance from the surface where as in point cloud we can only tell the occupancy probability.
* Memory usage is high in point clouds, where as high compression can be done in SDF as voxels nearer to the surface are stored (truncated).
* The distance value we get during SDF can be used for trajectory generation, which we can't in point clouds.

# References and Resources

1. Gimbal locks and quaternions: https://youtu.be/YF5ZUlKxSgE
2. Exponential map: 
    1. 3 Blue 1 Brown: https://youtu.be/O85OWBJ2ayo
    2. Northwestern Robotics: https://youtu.be/v_KBHaG0mas
3. Bunny ply is taken from: http://graphics.im.ntu.edu.tw/~robin/courses/cg03/model/